# Sesiones prácticas

## 1

Instalación, python, numpy

## 2

Dispositivos de captura


- webcam.py con opencv crudo
- spyder
- umucv (install con --upgrade)
- PYTHONPATH para sus módulos
- stream.py, opciones de autostream, efecto de teclas
- ejemplo de B/N invertido
- grabar video de demo
- servidor mjpeg,

## 3

- medidor.py
- inrange.py

## 4

- captura en hilo aparte
- mean shift / camshift

## 5

- hog/hog0.py

- hog/pedestrian.py con

- dlib: hog/facelandmarks.py

- dlib: herramienta imglab, hog/train_detector.py, hog/run_detector.py

## 6

- LK/corners0.py, 1, 2, 3, LK/lk_tracks.py

## 7

En esta sesión vamos a experimentar con el detector de puntos de interés SIFT. (La implementación de opencv está en un repositorio aparte con las contribuciones "non free", pero la patente ha expirado hace unos días. En cualquier caso, la versión de opencv que estamos usando lo incluye.)

Nuestro objetivo es calcular un conjunto de "keypoints", cada uno con su descriptor (vector de características que describe el entorno del punto), que nos permita encontrarlo en imágenes futuras. Esto tiene una aplicación inmediata para reconocer objetos y más adelante en geometría visual.

Empezamos con el ejemplo de código code/SIFT/sift0.py, que simplemente calcula y muestra los puntos de interés. Es interesante observar el efecto de los parámetros del método y el tiempo de cómputo en función del tamaño de la imagen (que puedes cambiar con --size o --resize).

El siguiente ejemplo code/SIFT/sift1.py muestra un primer ataque para establecer correspondencias. Los resultados son bastante pobres.

Finalmente, en code/SIFT/sift.py aplicamos un criterio de selección para eliminar muchas correspondencias erróneas (aunque no todas). Esto es en principio suficiente para el reconocimiento de objetos. (Más adelante veremos una forma mucho mejor de eliminar correspondencias erróneas, necesaria para aplicaciones de geometría.)

El ejercicio obligatorio **SIFT** es una ampliación sencilla de este código. Se trata de almacenar un conjunto de modelos (¡con textura! para que tengan suficientes keypoints) como portadas de libros, discos, videojuegos, etc. y reconocerlos en base a la proporción de coincidencias detectadas.

Una segunda actividad en esta sesión consiste en comentar el ejemplo de código code/server.py. Utiliza el paquete [flask][flask] para crear un sencillo servidor web que devuelve la imagen de la cámara modificada como deseemos. Sirve como punto de partida para el ejercicio opcional **WEB**.

[flask]: https://en.wikipedia.org/wiki/Flask_(web_framework)